## Pytorch Lightning Tutorial

In [50]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import MNIST

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import torchmetrics as tm

pl.seed_everything(61290)

Global seed set to 61290


61290

In [51]:
import wandb
wandb.login()

True

In [36]:
# Hyperparameter

in_dim = 784
out_dim = 10
n_hidden_1 = 256
n_hidden_2 = 128
num_epochs = 1
batch_size = 64
lr = 0.001


In [37]:
class BasicNet(pl.LightningModule):
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):
        super(BasicNet, self).__init__()
        self.layer1 = nn.Linear(in_dim, n_hidden_1)
        self.layer2 = nn.Linear(n_hidden_1, n_hidden_2)
        self.layer3 = nn.Linear(n_hidden_2, out_dim)

        self.train_accuracy = tm.Accuracy()
        self.val_accuracy = tm.Accuracy()

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return x

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=lr)

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = x.view(-1, 784)
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        x = x.view(-1, 784)
        y_hat = self(x)
        acc = self.val_accuracy(y_hat, y)
        self.log('val_acc', acc, on_step=True, on_epoch=True, prog_bar=True)
        return F.cross_entropy(y_hat, y)  # val loss


In [38]:
dataset = MNIST(root='./mnist',
                train=True,
                download=True,
                transform=transforms.ToTensor())


In [39]:
mnist_train, mnist_val = torch.utils.data.random_split(dataset, [55000, 5000])

In [40]:
train_loader = torch.utils.data.DataLoader(mnist_train,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=8)
val_loader = torch.utils.data.DataLoader(mnist_val,
                                         batch_size=batch_size,
                                         shuffle=False,
                                         num_workers=8)


In [41]:
model = BasicNet(in_dim, n_hidden_1, n_hidden_2, out_dim)


In [42]:
# from pytorch_lightning.loggers import TensorBoardLogger
# logger = TensorBoardLogger("./lightning_logs", name="my_model")

In [44]:
trainer = pl.Trainer(gpus=0, precision=32, max_epochs=num_epochs)

trainer.fit(model, train_loader, val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name           | Type     | Params
--------------------------------------------
0 | layer1         | Linear   | 200 K 
1 | layer2         | Linear   | 32.9 K
2 | layer3         | Linear   | 1.3 K 
3 | train_accuracy | Accuracy | 0     
4 | val_accuracy   | Accuracy | 0     
--------------------------------------------
235 K     Trainable params
0         Non-trainable params
235 K     Total params
0.941     Total estimated model params size (MB)


Epoch 0: 100%|██████████| 939/939 [00:19<00:00, 47.08it/s, loss=0.166, v_num=1, train_loss_step=0.179, val_acc_step=0.750, val_acc_epoch=0.959]


In [34]:
# test the model

model.eval()
accuracy = []

for x,y in val_loader:
    x = x.view(-1, 784)
    y_pred = model(x)
    cls = torch.argmax(y_pred, dim=1)
    acc = (y == cls).sum().item() / y.shape[0]
    accuracy.append(acc)
    
    
    

In [35]:
import numpy as np
accuracy = np.mean(accuracy)
accuracy

0.9810126582278481